<a href="https://colab.research.google.com/github/maxilamb/MARK/blob/master/TensorflowJS_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run cell (commands) below and then hit F5 (refresh the page) right after that

In [ ]:
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global
!jupyter-kernelspec list

That's it! :) Now in the next cells you can write JavaScript

# Example of usage

**Remember not to use `const` or `let`! Use `var` instead**

This is how you can execute shell commands:

In [ ]:
var { spawn } = require('child_process');
var sh = (cmd) => { 
    $$.async();
    var sp = spawn(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' });
    sp.stdout.on('data', data => console.log(data.toString()));
    sp.stderr.on('data', data => console.error(data.toString()));
    sp.on('close', () => $$.done());
};
var run_async = async (pf) => {
  $$.async();
  await pf();
  $$.done();
};
sh('npm init -y');

In [ ]:
sh('node -v; npm -v');

In [ ]:
sh('npm install @tensorflow/tfjs-node-gpu')

In [ ]:
var tf = require('@tensorflow/tfjs-node-gpu');
tf.backend().isUsingGpuDevice;